In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version


spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubun

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-16 19:59:15--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.79MB/s    in 0.2s    

2021-06-16 19:59:15 (5.79 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [ ]:
vine_df_full = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df_full.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RMDCHWD0Y5OZ9|          3|            0|          1|   N|                N|
| RZSL0BALIYUNU|          5|            0|          0|   N|                Y|
| RIZR67JKUDBI0|          3|            0|          1|   N|                Y|
|R27HL570VNL85F|          5|            0|          0|   N|                Y|
|R34EBU9QDWJ1GD|          5|            0|          0|   N|                Y|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|                N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|                Y|
|R3GYQ5W8JHP8SB|          5|            0|          0|   N|                Y|
|R30SHYQXGG5EYC|          5|            0|          0|   N|                Y|
|R14YLXA56NP51I|          5|            1|          1|   N|     

In [ ]:
# 1. filter the data where total_votes count is equal to or reater than 20 - prevent div/0 later and see what reviews are deemed 'useful'

vine_df = vine_df_full.filter('total_votes>=20')
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|  RMRD6SMF2AUQ|          3|            4|         21|   N|                N|
| RMPCXKWX3T57Y|          1|            1|         73|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|     

In [ ]:
# 2. filter the data where the number of helpful_votes divided by total_votes is >= 50%
helpful_vines = vine_df.withColumn('helpful_perc',(vine_df['helpful_votes'] / vine_df['total_votes'])).filter('helpful_perc>=0.50')
helpful_vines.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|      helpful_perc|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|0.7704918032786885|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|0.9130434782608695|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|0.9736842105263158|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|0.8918918918918919|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|0.9583333333333334|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y| 0.918918918918919|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|0.9565217391304348|
|R1P7GJ0IN2BRNH|          5|  

In [ ]:
# 3. split dataframe into two sets based on boolean vine column value, SQL style and pythonic, both give the same answer
helpful_vine_Y = helpful_vines.filter('vine="Y"') 
helpful_vine_N = helpful_vines.filter(helpful_vines['vine'] == 'N')

helpful_vine_Y.count(), helpful_vine_N.count()

(60, 14477)

In [ ]:
import pyspark.sql.functions as f

In [ ]:
# 4. determine the total number of reviews, number of 5-star reviews, and percentage of 5-star reviews for the two dataframes
help_Y_total = helpful_vine_Y.count()
help_Y_5star = helpful_vine_Y.filter('star_rating==5').count()
help_Y_5perc = (help_Y_5star / help_Y_total) * 100

cols = ['Total_Reviews','5Star_Reviews']
data = [(help_Y_total), (help_Y_5star)]

# Vine: Y

In [ ]:
help_y_sum = spark.createDataFrame([data],cols)
help_y_sum.withColumn('5_Star_Percent', (help_y_sum['5Star_Reviews']/help_y_sum['Total_Reviews'])*100).show()

+-------------+-------------+------------------+
|Total_Reviews|5Star_Reviews|    5_Star_Percent|
+-------------+-------------+------------------+
|           60|           34|56.666666666666664|
+-------------+-------------+------------------+



# Vine: N

In [ ]:
help_N_total = helpful_vine_N.count()
help_N_5star = helpful_vine_N.filter('star_rating==5').count()
help_N_5perc = (help_N_5star / help_N_total) * 100

cols = ['Total_Reviews','5Star_Reviews']
data = [(help_N_total), (help_N_5star)]

help_n_sum = spark.createDataFrame([data],cols)
help_n_sum.withColumn('5_Star_Percent', (help_n_sum['5Star_Reviews']/help_n_sum['Total_Reviews'])*100).show()

+-------------+-------------+-----------------+
|Total_Reviews|5Star_Reviews|   5_Star_Percent|
+-------------+-------------+-----------------+
|        14477|         8212|56.72445948746287|
+-------------+-------------+-----------------+



# Same Thing - With Pandas

In [ ]:
import pandas as pd
vine_pd = vine_df_full.toPandas()
vine_pd.head(10)

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
0,RMDCHWD0Y5OZ9,3.0,0.0,1.0,N,N
1,RZSL0BALIYUNU,5.0,0.0,0.0,N,Y
2,RIZR67JKUDBI0,3.0,0.0,1.0,N,Y
3,R27HL570VNL85F,5.0,0.0,0.0,N,Y
4,R34EBU9QDWJ1GD,5.0,0.0,0.0,N,Y
5,R1WCUI4Z1SIQEO,5.0,0.0,0.0,N,N
6,RL5LNO26GAVJ1,2.0,3.0,4.0,N,Y
7,R3GYQ5W8JHP8SB,5.0,0.0,0.0,N,Y
8,R30SHYQXGG5EYC,5.0,0.0,0.0,N,Y
9,R14YLXA56NP51I,5.0,1.0,1.0,N,N


In [ ]:
helpful_vine_pd = vine_pd[vine_pd['total_votes'] >= 20]
helpful_vine_pd = helpful_vine_pd[helpful_vine_pd['helpful_votes']/helpful_vine_pd['total_votes'] >= 0.50]
helpful_vine_pd.head(10)

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
28,R2243Y3OD8U6KQ,5.0,47.0,61.0,N,N
85,R2TGT0CDTCAAHW,5.0,21.0,23.0,N,Y
457,RX4D22YSXEF4P,1.0,37.0,38.0,N,Y
1028,R3FL2NTLFUSPTQ,5.0,33.0,37.0,N,N
1039,R3QTP3YNZXAPPF,3.0,23.0,24.0,N,Y
1140,R36V6V42VN5AS5,5.0,34.0,37.0,N,Y
1646,R27LZWE27BJPOB,5.0,22.0,23.0,N,N
2261,R1P7GJ0IN2BRNH,5.0,37.0,37.0,N,Y
2340,R2R6JPF9KOD2HJ,5.0,19.0,20.0,N,Y
2850,R2J0ZZGFXKM8KR,2.0,21.0,22.0,N,Y


In [ ]:
f"Pandas Step 2: {len(helpful_vine_pd):,}   Pyspark Step 2: {helpful_vines.count():,}"

'Pandas Step 2: 14,537   Pyspark Step 2: 14,537'

In [ ]:
helpful_pd_Y = helpful_vine_pd[helpful_vine_pd['vine'] == 'Y']
helpful_pd_N = helpful_vine_pd[helpful_vine_pd['vine'] == 'N']

f"{len(helpful_pd_Y)}, {len(helpful_pd_N)} ---- {helpful_vine_Y.count()}, {helpful_vine_N.count()}"

'60, 14477 ---- 60, 14477'

# Vine: Y (Pandas)


In [ ]:
pd_Y_output = [len(helpful_pd_Y), len(helpful_pd_Y[helpful_pd_Y['star_rating']==5]), len(helpful_pd_Y[helpful_pd_Y['star_rating']==5])/len(helpful_pd_Y)]
pd_N_output = [len(helpful_pd_N), len(helpful_pd_N[helpful_pd_N['star_rating']==5]), len(helpful_pd_N[helpful_pd_N['star_rating']==5])/len(helpful_pd_N)]
columns = ['Total','5-Star','Perc_5-Star']

In [ ]:
pd.DataFrame({columns[i]:pd_Y_output[i] for i in range(3)}, index=[""])

,Total,5-Star,Perc_5-Star
,60,34,0.566667


#Vine: N (Pandas)

In [ ]:
pd.DataFrame({columns[i]:pd_N_output[i] for i in range(3)}, index=[""])

,Total,5-Star,Perc_5-Star
,14477,8212,0.567245


In [ ]:
from google.colab import files
vine_pd.to_csv('Vine_DF.csv', index=False)
files.download("Vine_DF.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>